In [12]:
import numpy as np
import tensorflow as tf
AUTOTUNE = tf.data.experimental.AUTOTUNE
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Input
import cv2
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.models import Model
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.utils.layer_utils import count_params
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os 
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D, BatchNormalization
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from keras.applications import MobileNet
from keras.callbacks import ModelCheckpoint
from keras.applications import VGG16
from tensorflow.keras.preprocessing.image import load_img, img_to_array
%matplotlib inline

In [8]:
train_path = 'train'
val_path = 'validation'
test_path = 'test'

In [9]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=30,
                                   zoom_range=0.2)

validation_datagen = ImageDataGenerator(rescale=1/255)
batch_size = 128

train_generator = train_datagen.flow_from_directory (train_path,   
                                                     target_size=(48, 48),  
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     class_mode='categorical',
                                                     color_mode='rgb')

validation_generator = validation_datagen.flow_from_directory(val_path,  
                                                              target_size=(48,48), 
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              color_mode='rgb')

Found 28750 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [13]:
#Loading the Mobilenet model 
featurizer = VGG16(include_top=False, weights='imagenet', input_shape=(48,48,3))

#Since we have 7 types of expressions, we'll set the nulber of classes to 7
num_classes = 7

#Adding some layers to the feturizer
x = Flatten()(featurizer.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
predictions = Dense(num_classes, activation = 'softmax')(x)



model = Model(input = featurizer.input, output = predictions)

for layer in model.layers[:-4]:
    layer.trainable = False

model.compile(Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])


model.summary()

AttributeError: module 'h5py' has no attribute 'File'

In [ ]:
#model_weights_path = r'/content/drive/My Drive/DH/SEGUNDO PROYECTO/model_weights.h5'

#checkpoint = ModelCheckpoint(model_weights_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')


history = model.fit(train_generator,
                        steps_per_epoch=train_generator.n//train_generator.batch_size,
                        validation_steps=validation_generator.n//validation_generator.batch_size,
                        epochs=10,
                        verbose=1,
                        validation_data = validation_generator,
                        callbacks=[checkpoint])